In [ ]:
# Only run this if ipywidgets is not installed
# !pip install ipywidgets

In [ ]:
import pandas as pd

from IPython.display import display
from ipywidgets import Button, Textarea, HBox, VBox, Label, HTML, ToggleButtons, BoundedIntText
from datetime import datetime
import numpy as np

In [ ]:
ROOT = "~/Desktop/loop_study_grader/coding save/"

In [ ]:
df = pd.read_excel(ROOT + "grading_results_15-Jan-2022_23-48.xlsx")
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

In [ ]:
# Add rubric columns only if not available
if 'Q1_strategy' not in df.columns:
    for i in range(1, 6):
        df[f'Q{i}_strategy'] = None
        df[f'Q{i}_iteration'] = None
        df[f'Q{i}_hard_coded'] = None
        df[f'Q{i}_correctness'] = None
else:
    # convert nan to None as widget could not take nan as input value
    df=df.where(df.notnull(), None)

In [ ]:
df.head(3)

In [ ]:
num_to_option = {1: "Yes", 0: "No"}
option_to_num = {"Yes": 1, "No": 0}

strategy_rubric = "Which loop strategy is used?"
iteration_rubric = "Is number of iterations correct?"
hard_coded_rubric = "Is the loop hard coded?"
correctness_rubric = "Is the solution correct (ignoring minor syntax errors)?"

for i in range(1, 6):
    df.at[0, f'Q{i}_strategy'] = strategy_rubric
    df.at[0, f'Q{i}_iteration'] = iteration_rubric
    df.at[0, f'Q{i}_hard_coded'] = strategy_rubric
    df.at[0, f'Q{i}_correctness'] = correctness_rubric

In [ ]:
# Add question title button
# We wrap it in a button so that we can display the tooltip
question_button = Button(
    tooltip="tooltip", 
    disabled=True, 
    button_style="info",
    layout={"width": "99%"}
)

# The text area for displaying students' solutions
solution_ta = Textarea(
    layout={"height": "500px", "width": "99%"},
)

# The navigation buttons for questions
question_prev_button = Button(description="Previous Question", layout={"width": "49%"})
question_next_button = Button(description="Next Question", layout={"width": "49%"})
question_button_box = HBox(
    [question_prev_button, question_next_button],
    layout={"margin": "20px 0px 0px 0px"}
)

# Group all UIs on the left side into a box
left_box = VBox(
    [question_button, solution_ta, question_button_box],
    layout={"height": "600px", "width": "50%"},
)


# ---------------- The right side -------------------

# The label and toggle for coding the strategy
strategy_label = Label(strategy_rubric)
strategy_toggle_button = ToggleButtons(
    value=None,
    options=["For-loop", "While-loop"],
    layout={"margin": "0px 0px 20px 0px"}
)

# The label and toggle for coding the iteration numbers
iteration_label = Label(iteration_rubric)
iteration_toggle_button = ToggleButtons(
    value=None,
    options=["Yes", "No"],
    layout={"margin": "0px 0px 20px 0px"},
)

# The label and toggle for coding the hard coded loop
hard_coded_label = Label(hard_coded_rubric)
hard_coded_toggle_button = ToggleButtons(
    value=None,
    options=["Yes", "No"],
    layout={"margin": "0px 0px 20px 0px"},
)

# The label and toggle for coding the correctness
correctness_label = Label(correctness_rubric)
correctness_toggle_button = ToggleButtons(
    value=None,
    options=["Yes", "No"],
    layout={"margin": "0px 0px 20px 0px"}
)

# Group all rubric widgets into a vertial box
rubric_box = VBox([
    strategy_label, strategy_toggle_button, 
    iteration_label, iteration_toggle_button, 
    hard_coded_label, hard_coded_toggle_button,
    correctness_label, correctness_toggle_button],
    layout={
        "height": "400px",
        "margin": "0px 0px 0px 0px"
    }
)

# Create a bounded text box for jumping to different students
jump_bounded = BoundedIntText(value=1, min=1, max=len(df) - 1, step=1, description="Jump to: ")
jump_button = Button(description="Go")
jump_box = HBox([jump_bounded, jump_button], layout={"width": "80%", "margin": "0px 0px 10px 0px"})

# Student nagivation buttons
student_prev_button = Button(description="Previous Student", layout={"width": "39%"})
student_next_button = Button(description="Next Student", layout={"width": "39%"})
student_box = HBox([student_prev_button, student_next_button])

# Save button
save_button = Button(
    description="Save", 
    button_style='Success',
    layout={"width": "80%", "margin": "10px 0px 0px 0px"}
)
save_label = HTML("")

# Reset button
reset_button = Button(
    description="Reset",
    button_style="danger",
    layout={"width": "80%", "margin": "20px 0px 0px 0px"}
)

# Group all widgets on the right side into a box
right_box = VBox([
    rubric_box,
    jump_box,
    student_box, 
    reset_button,
    save_button, 
    save_label,
    ],
    layout={
        "height": "500px", 
        "width": "50%",
        "margin": "0px 0px 0px 20px"
    },
)

ui = HBox([left_box, right_box], layout={"width": "900px", "margin": "50px"})

In [ ]:
# Global variables to track the index
sidx = 1  # student index  
qidx = 1  # question index

# ---------------- Define some actions -------------

def refresh_ui():
    # Question button
    question = f'Q{qidx}'
    question_button.description = question
    question_button.tooltip = df.iloc[0][question]
    
    # Solution text area
    solution = df.iloc[sidx][question]
    solution = solution.replace('&lt;', '<').replace('&gt;', '>')
    solution_ta.value = solution
    
    # Question navigation buttons
    question_prev_button.disabled = True if qidx == 1 else False
    question_next_button.disabled = True if qidx == 5 else False
    
    # Toggles
    strategy_toggle_button.value = df.iloc[sidx][f'Q{qidx}_strategy']
    iteration_toggle_button.value = num_to_option.get(df.iloc[sidx][f'Q{qidx}_iteration'])
    hard_coded_toggle_button.value = num_to_option.get(df.iloc[sidx][f'Q{qidx}_hard_coded'])
    correctness_toggle_button.value = num_to_option.get(df.iloc[sidx][f'Q{qidx}_correctness'])
    
    # Update the current student (row number)
    jump_bounded.value = sidx
    
    student_prev_button.disabled = True if sidx == 1 else False
    student_next_button.disabled = True if sidx == len(df) - 1 else False
    
    save_label.value = ""

def save_current_result():
    df.at[sidx, f'Q{qidx}_strategy'] = strategy_toggle_button.value
    df.at[sidx, f'Q{qidx}_iteration'] = option_to_num.get(iteration_toggle_button.value)
    df.at[sidx, f'Q{qidx}_hard_coded'] = option_to_num.get(hard_coded_toggle_button.value)
    df.at[sidx, f'Q{qidx}_correctness'] = option_to_num.get(correctness_toggle_button.value)
    
def on_navigation_button_click(b):
    global qidx
    global sidx
    
    save_current_result()
    
    if b.description == "Previous Student":
        sidx -= 1
    elif b.description == "Next Student":
        sidx += 1
    elif b.description == "Previous Question":
        qidx -= 1
    else:
        qidx += 1
    
    refresh_ui()

for button in [
    question_prev_button, question_next_button, 
    student_prev_button, student_next_button
]:
    button.on_click(on_navigation_button_click)
    
def save_button_click(obj):
    save_current_result()
    out_name = datetime.now().strftime("grading_results_%d-%b-%Y_%H-%M.xlsx")
    out_name = ROOT + out_name
    save_label.value = f"A new copy is saved to <b>{out_name}</b>"
    df.to_excel(out_name)
    
save_button.on_click(save_button_click)

def jump_button_click(obj):
    global sidx
    save_current_result()
    sidx = jump_bounded.value
    refresh_ui()

jump_button.on_click(jump_button_click)

def reset_button_click(obj):
    for toggle in [
        strategy_toggle_button,
        iteration_toggle_button,
        hard_coded_toggle_button,
        correctness_toggle_button
    ]:
        toggle.value = None

reset_button.on_click(reset_button_click)
    
display(ui)
refresh_ui()